In [ ]:
from google.colab import drive
import torch
import torchvision
import torch.nn
import glob,pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM


In [ ]:
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
midi = converter.parse("/gdrive/My Drive/chopin/format/chpn-p9_format0.mid")

In [ ]:
#Notes(음정, 박자를 포함하는 정보)
notes=[]

In [ ]:
#데이터 불러오기 및 전처리
for i, file in enumerate(glob.glob("/gdrive/My Drive/chopin/format/*.mid")):
  midi=converter.parse(file)
  print('\r','Parsing file',i," ",file, end='') #진행상황 출력문
  print()
    
  notes_to_parse=None #MIDI -> Notes
    
  #MIDI 파일의 Note / Chord / Tempo 정보
  #파트의 구조 전처리
  try:
    s2=instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse()
    
  except: 
    notes_to_parse = midi.flat.notes
    

  #Note와 Chord의 음높이(pitch) 전처리
  for e in notes_to_parse:
    if isinstance(e, note.Note):
        notes.append(str(e.pitch))
            
    elif isinstance(e, chord.Chord):
        notes.append('.'.join(str(n) for n in e.normalOrder))

 Parsing file 0   /gdrive/My Drive/chopin/format/chpn-p21_format0.mid
 Parsing file 1   /gdrive/My Drive/chopin/format/chpn-p5_format0.mid
 Parsing file 2   /gdrive/My Drive/chopin/format/chpn-p4_format0.mid
 Parsing file 3   /gdrive/My Drive/chopin/format/chpn-p14_format0.mid
 Parsing file 4   /gdrive/My Drive/chopin/format/chpn-p13_format0.mid
 Parsing file 5   /gdrive/My Drive/chopin/format/chpn-p15_format0.mid
 Parsing file 6   /gdrive/My Drive/chopin/format/chpn-p19_format0.mid
 Parsing file 7   /gdrive/My Drive/chopin/format/chpn-p20_format0.mid
 Parsing file 8   /gdrive/My Drive/chopin/format/chpn-p11_format0.mid
 Parsing file 9   /gdrive/My Drive/chopin/format/chpn-p16_format0.mid
 Parsing file 10   /gdrive/My Drive/chopin/format/chpn-p17_format0.mid
 Parsing file 11   /gdrive/My Drive/chopin/format/chpn-p12_format0.mid
 Parsing file 12   /gdrive/My Drive/chopin/format/chpn-p18_format0.mid
 Parsing file 13   /gdrive/My Drive/chopin/format/chpn-p1_format0.mid
 Parsing file 14   

In [ ]:
#MIDI 파일 정보를 다루기 쉽게 바꿔줌
#여기서 쓰는 각각의 음정은 총 Classes of notes 개로 1~len(notes)에 음이 할당

n_vocab = (len(set(notes)))
print('Classes of notes: ', n_vocab, '\n') #노트들의 총 가짓수
print('notes: ', notes[:500])
print('length of notes: ', len(notes), '\n')

#모든 Note/Chord를 정렬해놓은 배열
pitchnames = sorted(set(item for item in notes))
print('pitchnames: ', pitchnames)
print('length of pitchnames: ', len(pitchnames), '\n')

#음높이(pitch)를 정수에 매핑
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print('note_to_int: ', note_to_int)

Classes of notes:  311 

notes:  ['F5', 'B-1', 'F3', '4.7', '3.9', '10.2', '0', 'D5', 'B-2', '2.5', '4.7', '3.9', 'F5', 'E-5', '10.2', 'B-2', 'D5', 'E-2', 'G3', '6.9', '5.11', '0.3', '2', 'G4', 'C3', '3.7', '6.9', '5.11', 'E-4', '0.3', 'C3', 'A4', 'F2', '6.10', 'E-5', '5.9', '8.0', 'E-5', '3.7', '6.10', 'E-5', 'F2', '3.6', 'F5', 'E-5', 'D5', '2.5', '4.7', 'E-5', '3.9', 'F3', 'G5', 'B-2', 'F3', '4.7', '3.9', '10.2', '0', 'F5', 'D5', 'B-2', '2.5', '4.7', '3.9', '10.2', '0', 'B-5', 'G2', 'D4', '1.4', '0.6', '7.10', '9', 'B-4', 'B-2', 'B-3', '9.0', '2.8', '3.7', '5', 'B-4', 'G5', 'E-2', 'G3', '6.9', '5.11', '0.3', '2', '7.0', 'G4', '3.7', 'A4', 'G4', 'F#4', 'G4', 'E-5', '6.9', '5.11', 'C5', '0.3', '0.3', '5.9', '2.5', '4.7', '3.9', '10.2', '1.5', '8.0', '6.10', '9.0', '2.8', '3.7', '6.10', 'B-2', '2.5', '4.7', '3.9', '10.2', '5', '10', '2.5', '4.7', '3.9', '10.2', 'B-2', '10.1', 'B-1', '10.1', '1.6', '10.3', '1.6', '10.1', '6.10', '1.6', '10.3', '1.6', '10.1', '11.3', '1.6', '10.3', '10.1'

In [ ]:
#LSTM 모델 Training Dataset 생성
#작곡이라 validation이 필요 없음

seq_len = 100

#pitch를 정수로 바꿔서 LSTM 입출력으로 만든다.
net_in=[] #입력
net_out=[] #출력

#100개를 보고 예측할 seq_out을 구성
for i in range(0, len(notes) - seq_len):
  seq_in=notes[i:i+seq_len] #[0~99]
  seq_out=notes[i+seq_len] #[100]

  #문자열 숫자 -> 정수
  net_in.append([note_to_int[char] for char in seq_in])
  net_out.append([note_to_int[seq_out]])
    
print(np.shape(net_in))
print(np.shape(net_out))

(21344, 100)
(21344, 1)


In [ ]:
#Dataset 전처리
#100개 적은 패턴 생성
n_patterns = len(net_in)
print('n_patterns: ', n_patterns)

#입력 모양: 샘플 수, 시퀀스 길이, 자료의 차원
net_in=np.reshape(net_in, (n_patterns, seq_len, 1))
print('shape of net_in: ', net_in.shape)

#데이터 범위 정규화
net_in=net_in/float(n_vocab)

#One-hot (분류)
net_out=np_utils.to_categorical(net_out)
print('shape of net_out: ', net_out.shape)

n_patterns:  21344
shape of net_in:  (21344, 100, 1)
shape of net_out:  (21344, 311)


In [ ]:
#모델 구성

data_dim=net_in.shape[2]

model=Sequential(name="Chopin_LSTM")
model.add(LSTM(256, input_shape=(seq_len, data_dim), return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "Chopin_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 256)          264192    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          1574912   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_2 (LSTM)               (None, 256)               787456    
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 311)               

In [ ]:
model.fit(net_in, net_out, epochs=200, batch_size=64)

Epoch 1/200
334/334 [==============================] - 62s 185ms/step - loss: 2.0212
Epoch 2/200
334/334 [==============================] - 62s 186ms/step - loss: 1.6839
Epoch 3/200
334/334 [==============================] - 62s 186ms/step - loss: 1.3815
Epoch 4/200
334/334 [==============================] - 62s 186ms/step - loss: 1.1282
Epoch 5/200
334/334 [==============================] - 62s 186ms/step - loss: 0.8962
Epoch 6/200
334/334 [==============================] - 62s 185ms/step - loss: 0.7374
Epoch 7/200
334/334 [==============================] - 62s 186ms/step - loss: 0.5677
Epoch 8/200
334/334 [==============================] - 62s 186ms/step - loss: 0.4334
Epoch 9/200
334/334 [==============================] - 62s 186ms/step - loss: 0.3388
Epoch 10/200
334/334 [==============================] - 62s 186ms/step - loss: 0.2733
Epoch 11/200
334/334 [==============================] - 62s 186ms/step - loss: 0.2213
Epoch 12/200
334/334 [==============================] - 62s 186

In [ ]:
#작곡을 위해 LSTM 모델 입력을 다시 만든다.
net_in=[] #입력
output=[] #출력

#LSTM 모델의 입출력을 만들기 위해 (전체 길이 - 시퀀스길이(=100))만큼 반복
#ex) 입력: 출력 짝지어주기
for i in range(0, len(notes) - seq_len):
  seq_in=notes[i:i+seq_len] #[0~99]
  seq_out=notes[i+seq_len] #[100]

  #문자열을 정수로 변환
  net_in.append([note_to_int[char] for char in seq_in])
  output.append([note_to_int[seq_out]])

n_patterns=len(net_in)

In [ ]:
#작곡 시작점을 랜덤한 시퀀스로 고름
start=np.random.randint(0, len(net_in)-1)
pattern=net_in[start]
print('Random Sequence: ', pattern)

#int_to_note 정수를 다시 Note로 바꾸기 위한 dictionary 자료형
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
print('int_to_note: ', int_to_note)

Random Sequence:  [245, 199, 251, 126, 251, 108, 263, 214, 263, 257, 126, 257, 199, 270, 126, 270, 108, 276, 66, 276, 66, 283, 245, 283, 123, 276, 66, 276, 66, 257, 126, 257, 126, 251, 126, 251, 82, 257, 126, 257, 126, 276, 142, 276, 203, 257, 142, 257, 23, 251, 35, 251, 203, 245, 256, 123, 251, 251, 256, 108, 245, 245, 256, 123, 275, 214, 275, 282, 250, 282, 199, 295, 82, 295, 108, 289, 214, 289, 308, 82, 308, 199, 302, 82, 302, 108, 239, 140, 239, 245, 86, 245, 182, 251, 86, 251, 128, 263, 80, 263]
int_to_note:  {0: '0', 1: '0.1.3.7', 2: '0.1.4.8', 3: '0.1.5', 4: '0.2', 5: '0.2.4', 6: '0.2.5', 7: '0.2.6', 8: '0.2.6.8', 9: '0.3', 10: '0.3.5', 11: '0.3.5.6', 12: '0.3.5.8', 13: '0.3.6', 14: '0.3.6.8', 15: '0.3.6.9', 16: '0.3.7', 17: '0.4', 18: '0.4.6', 19: '0.4.7', 20: '0.4.8', 21: '0.5', 22: '0.6', 23: '1', 24: '1.2', 25: '1.2.6.9', 26: '1.3', 27: '1.3.4', 28: '1.3.5', 29: '1.3.5.7.10', 30: '1.3.5.8', 31: '1.3.6', 32: '1.3.6.8', 33: '1.3.7', 34: '1.3.8', 35: '1.4', 36: '1.4.6', 37: '1.

In [ ]:
# 출력값 리스트
pred_out = []

# 500 note 생성
for i in range(0, 500):
  pred_in = np.reshape(pattern, (1, len(pattern), 1))
  
  # 입력 범위 정규화 / 0 ~ (n_vocab -1) => 0 ~ 1  
  pred_in = pred_in / float(n_vocab)

  # 예측
  prediction = model.predict(pred_in, verbose=0)

  # 출력 중 값이 가장 큰 Index 선택
  index = np.argmax(prediction)

  # 정수 값을 Note 값으로 변경
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')

  pred_out.append(result)

  # 새 값 추가 후 과거 값 제거
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

 Predicted  499   C#3

In [ ]:
#LSTM이 출력한 500개의 음정
print('length of pred_out : ', len(pred_out))
print('pred_out : ', pred_out)

length of pred_out :  500
pred_out :  ['C#5', 'F4', 'C#5', '7', 'D5', '5.11', '7', 'D5', '6.11', 'E-5', '6', 'E5', '6.11', 'E5', '3.6', 'E-5', '6.11', 'E-5', '6', 'F#5', '6.10', 'F#5', '4.6', 'C#5', '6.10', 'C#5', '6', 'E-5', '6.10', 'E-5', '4.6', 'E5', '6.11', 'E5', '6', 'E-5', '6.11', 'E-5', '3.6', 'G#5', 'F#4', 'G#5', '6', 'F#5', '6.10', 'F#5', '4.6', 'C#5', '4.10', 'C#5', '6', 'E-5', '4.10', 'E-5', '1.6', 'E5', '11.3', 'E5', '6.11', 'E-5', 'E-4', 'E-5', '6.11', 'B4', 'E4', 'B4', '7.11', 'G4', 'E-4', 'G4', '6.11', 'F#4', 'E-4', 'F#4', '6.11', 'B4', 'E4', 'B4', '7.11', 'G4', 'E-4', 'F#4', '6.11', 'B4', 'E4', 'B4', '4.7.11', 'G4', 'E-4', 'F#4', '6.11', 'B4', 'E4', 'B4', '4.7.11', 'G4', 'D4', 'F#4', '6.11', 'B4', 'E4', 'B4', '4.7.11', 'G4', 'B4', '4.7', '6.11', '2.6', 'G4', '2.6', '11', '1.4', '2.6', 'B2', '4.7', '6.11', '2.6', 'G4', '2.6', '11', '1.4', '11.2.6', '11.2.4', 'G#4', '0.4', 'G#4', '0.4', '11.2', 'G#4', '11.2.4', '0.4', 'A4', '2.5', '4.9', '0.4', 'F4', '0.4', '9', '11.2', '

In [ ]:
# MIDI 파일 생성
offset = 0 # 오프셋

# MIDI 파일
output_notes = []
l
# 예측 값 전처리
for pattern in pred_out:
    
  # pattern이 Chord 일 때
  if ('.' in pattern) or pattern.isdigit():
    notes_in_chord = pattern.split('.') # ['8.1'].split('.') => ['8', '1']
    notes = [] # Note 정보
      
    # Text => 정수 => Note
    for current_note in notes_in_chord:
        new_note = note.Note(int(current_note)) 
        new_note.storedInstrument = instrument.Piano() # 피아노로 설정
        notes.append(new_note)
    
    # Note => Chord
    new_chord = chord.Chord(notes)
    new_chord.offset = offset # 시간 정보 설정
    output_notes.append(new_chord)
      
  # pattern이 Note 일 때
  else:
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

  offset += 0.5
    
# Note/Chord => Stream => MIDI File
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/gdrive/My Drive/midi_result/output_midi_colab(200).mid')

'/gdrive/My Drive/midi_result/output_midi_colab(200).mid'

In [ ]:
from music21 import *
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = '/usr/bin/mscore'
us['directoryScratch'] = '/tmp'